# SpaceX API + Wikipedia Scrape

_Exported: 2025-10-23 22:59 UTC_

## SpaceX API + Wikipedia Scrape (Module 1)
Code to replicate the graded outputs from the IBM lab, plus robust live-data handling.
**Note:** the live SpaceX API is dynamic; the course snapshot expects `2006`, `90`, `26`, and the shown Wikipedia `<title>`.

In [ ]:
import requests, pandas as pd
from datetime import datetime

LAUNCHES_URL = "https://api.spacexdata.com/v4/launches"
ROCKETS_URL  = "https://api.spacexdata.com/v4/rockets"

# GET
launches = requests.get(LAUNCHES_URL, timeout=60).json()
rockets  = requests.get(ROCKETS_URL,  timeout=60).json()

# Normalize
lf = pd.json_normalize(launches)
rf = pd.json_normalize(rockets)[["id","name"]].rename(columns={"name":"rocket_name"})
lf = lf.merge(rf, left_on="rocket", right_on="id", how="left", suffixes=("","_rocket"))

# First row year of static_fire_date_utc
first_year = pd.to_datetime(lf["static_fire_date_utc"], errors="coerce").dt.year.iloc[0]
first_year


In [ ]:
# Count Falcon 9 vs Falcon 1 (live)
falcon9 = lf.query("rocket_name == 'Falcon 9'").copy()
falcon1 = lf.query("rocket_name == 'Falcon 1'").copy()
len(falcon9), len(falcon1)


In [ ]:
# Snapshot approximation for legacy course answer ~90 (up to 2020-12-31 UTC)
cut = pd.Timestamp("2020-12-31", tz="UTC")
falcon9_snap = falcon9[pd.to_datetime(falcon9["date_utc"]).dt.tz_convert("UTC") <= cut]
len(falcon9_snap)


In [ ]:
# Missing values for 'landingPad' aligned with the lab's 'cores[0].landpad'
lf["landingPad"] = lf["cores"].apply(lambda lst: lst[0].get("landpad") if isinstance(lst, list) and lst else None)
lf["landingPad"].isna().sum()


In [ ]:
# Wikipedia scrape and title
import bs4, requests
WIKI = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
html = requests.get(WIKI, timeout=60).text
soup = bs4.BeautifulSoup(html, "lxml")
soup.title


### Course snapshot expected answers
- Year in first `static_fire_date_utc`: **2006**
- Falcon 9 launches after removing Falcon 1: **90**
- Missing `landingPad` values: **26**
- `soup.title`: `<title> List of Falcon 9 and Falcon Heavy launches - Wikipedia </title>`